## DESCRIPTION

This notebook generates the data and prepares the variables

***
### SETUP

The script common_setup.py is called below.

You can modify the script at will if needed (e.g. add new packages, etc.)

If you want/need to preprocess the data, modify the flag to RUN_PREPROCESS=True.

In [1]:
from utils.common_setup import *

***
### GENERATE DATA

In [ ]:
if RUN_PREPROCESS:
    operation_meta = pd.read_csv(os.path.join(ROOT_PATH, "ops_meta.csv"))

    d = pd.read_csv(os.path.join(ROOT_PATH, "raw_dev.csv"))
                 
    d['death_date'] = pd.to_datetime(d['death_date'], format="mixed")
    d['creation_date'] = pd.to_datetime(d['creation_date'], format="mixed")
    i = 0
    for season in SEASONS_INCLUDED:
        for ops_id in d.operation_id.unique():
            ops = operation_meta.loc[operation_meta['id']==ops_id]
            
            season_end = date(season, END_SEASON_MONTH, END_SEASON_DAY)
            last_season_end = date(season-1, END_SEASON_MONTH, END_SEASON_DAY)

            start_month = ops['season_start_month'].values[0]
            start_day = ops['season_start_day'].values[0]
            
            season_start = date(season,start_month,start_day)
            next_season_start = date(season+1,start_month,start_day)
        
            state_logs = d.loc[d['operation_id']==ops_id]
            state_logs["death_date"]= state_logs["death_date"].fillna(next_season_start)
            state_logs["death_date"] = pd.to_datetime(state_logs["death_date"]).dt.tz_localize(None)
            state_logs["creation_date"] = pd.to_datetime(state_logs["creation_date"]).dt.tz_localize(
                None
            )
           
            created_during_season = (state_logs["creation_date"].dt.date < season_end) 
            # Died after the season start but was alive before
            died_after_season_start = state_logs["death_date"].dt.date >= season_start
            # Died before the next season 
            died_before_next_season_start = state_logs["death_date"].dt.date <= next_season_start + timedelta(days=BUFFER)
            # Exclude hive created betwween season start and that die before also 
            didnt_see_summer = ~((state_logs.apply(lambda x: x["death_date"].month in [1, 2, 3, 4, 5] , axis=1)) & (state_logs["creation_date"].dt.date > season_end))
        
            
            subset_state = state_logs.loc[created_during_season & died_after_season_start & died_before_next_season_start & didnt_see_summer]
            subset_state['season'] = season
            # Also replace death date > then end_of_exp because otherwise bias model (age)
            subset_state.loc[pd.to_datetime(subset_state['death_date']).dt.date > next_season_start, 'death_date'] = next_season_start

            subset_state['season_start_month'] = start_month 
            subset_state['season_start_day'] = start_day
            
            if i == 0:
                data = subset_state 
                i = 1 
            else:
                data = pd.concat([data,subset_state], axis=0)
    data.reset_index(drop=True, inplace=True)
    data.to_csv(os.path.join(ROOT_PATH, "merged_dev.csv"), index=False)

***
### LOADING DATA

In [3]:
if RUN_PREPROCESS:
    aqi = pd.read_csv(os.path.join(ROOT_PATH, "aqi_dev.csv"))
    weather = pd.read_csv(os.path.join(ROOT_PATH, "weather_dev.csv"))
    ee = pd.read_csv(os.path.join(ROOT_PATH, "ee_dev.csv"))
    ee.drop("ee-image", axis=1, inplace=True)
    ee.dropna(subset=["ndvi"], inplace=True)
    
    mov = pd.read_csv(os.path.join(ROOT_PATH, "mouvements.csv"))

    data = pd.read_csv(os.path.join(ROOT_PATH, "merged_dev.csv"))
else:
    data = pd.read_csv(os.path.join(ROOT_PATH, "preprocess.csv"))
data['death_date'] = pd.to_datetime(data['death_date'], format='mixed')
data = data.loc[data['season'].isin(SEASONS_INCLUDED)]

***
### DATA INSPECTION

In [4]:
if RUN_PREPROCESS:
    print(weather.isnull().mean())
# For some reason atmost pressure is not getting scrapped 

In [5]:
if RUN_PREPROCESS:
    print(aqi.isnull().mean())

In [6]:
if RUN_PREPROCESS:
    print(ee.isnull().mean())
# 2023 is null , need to update for simulations later...

In [7]:
if RUN_PREPROCESS:
    print(ee.head())

Add creation date and death date

In [8]:
if RUN_PREPROCESS:
    data['creation_date'] = pd.to_datetime(data['creation_date'], format='mixed').dt.date
    data['death_date']= pd.to_datetime(data['death_date']).dt.date

In [9]:
if RUN_PREPROCESS:
    print(Counter(data.hive_identity_id.value_counts()))
# Meaning that 16 632 hives are present in two season while 38405 are only in one season 

Label creation (based on season start end)

In [10]:
if RUN_PREPROCESS:
    
    def get_state_for_season(state_log):
        
        return state_log.apply(
            lambda x: (x['death_date'] + timedelta(days=BUFFER)) <= date(x['season'] + 1, x['season_start_month'], x['season_start_day']),
            axis=1
        )
    
    def get_age_for_season(state_log):
        return state_log.apply(
            lambda x : abs((x['death_date'] - x['creation_date']).days),
            axis=1
        )
    
    data['death_next_season'] = get_state_for_season(data)
    data['hive_age_next_season'] = get_age_for_season(data)

In [13]:
round(data['death_next_season'].mean(), 3)

0.355

Column types

In [14]:
data.dtypes

hid                              int64
creation_date                   object
operation_id                     int64
death_date              datetime64[ns]
season                           int64
season_start_month               int64
season_start_day                 int64
death_next_season                 bool
hive_age_next_season             int64
aqhi_average                   float64
prcp_average                   float64
wspd_average                   float64
ndvi_average                   float64
tavg_average                   float64
o3_average                     float64
aqhi_skew                      float64
prcp_skew                      float64
wspd_skew                      float64
ndvi_skew                      float64
tavg_skew                      float64
o3_skew                        float64
aqhi_max                       float64
prcp_max                       float64
wspd_max                       float64
ndvi_max                       float64
tavg_max                 

Sample size per operation ID

In [15]:
data.operation_id.value_counts()

operation_id
193    41419
69     28613
153    19587
55     12354
167     7157
161     6881
87      5658
207     3613
177     1941
210     1610
195     1298
194      853
220      612
212      553
219      341
160      314
224      251
159      215
205      194
83       115
204        5
Name: count, dtype: int64

Season sample size

In [16]:
data.season.value_counts()

season
2023    102475
2022     24403
2021      6706
Name: count, dtype: int64

BAD OPS in 2021-2022

In [17]:
if RUN_PREPROCESS:
    data = data.loc[~((data['operation_id'] == 51) & (data['season']==2021))]
    data = data.loc[~((data['operation_id'] == 87) & (data['season']==2021))]
    data = data.loc[~((data['operation_id']== 167) & (data['season']==2022))]

Average deaths and ages at next season by operation ID and season

In [18]:
data.drop(['death_date', 'creation_date'], axis=1) \
    .groupby(['season', 'operation_id']) \
    .mean(numeric_only=True)[['death_next_season', 'hive_age_next_season']]

death_next_season  hive_age_next_season
season operation_id                                         
2021   55                     0.316870            303.664862
       69                     0.778855            352.086344
       83                     0.000000            277.000000
2022   55                     0.189252            530.274366
       69                     0.458318            357.938778
       83                     0.823529            336.882353
       87                     0.678105            256.837119
       153                    0.170213            371.130785
       159                    0.525000            258.187500
       160                    0.333333            251.757576
       161                    0.376853            181.977765
       177                    0.005089             93.740458
2023   55                     0.379773            679.800366
       69                     0.608972            503.032304
       83                     0.155556            743.111111
       87                     0.361939            373.476734
       153                    0.189475            533.823926
       159                    0.629630            372.288889
       160                    0.612903            417.487903
       161                    0.112736            439.982446
       167                    0.387453            188.097527
       177                    0.172294            379.661472
       193                    0.332867            271.111881
       194                    0.051583            154.887456
       195                    0.057011            162.412943
       204                    0.800000            237.200000
       205                    0.005155            203.247423
       207                    0.450872            129.685026
       210                    0.075155            243.935404
       212                    0.602170            101.043400
       219                    0.067449             97.475073
       220                    0.302288            183.339869
       224                    0.000000            178.940239

In [21]:
len(data)

133584

In [22]:
data.isnull().mean()

hid                     0.000000
creation_date           0.000000
operation_id            0.000000
death_date              0.000000
season                  0.000000
season_start_month      0.000000
season_start_day        0.000000
death_next_season       0.000000
hive_age_next_season    0.000000
aqhi_average            0.072524
prcp_average            0.072524
wspd_average            0.072524
ndvi_average            0.072524
tavg_average            0.072524
o3_average              0.072524
aqhi_skew               0.076431
prcp_skew               0.079680
wspd_skew               0.076109
ndvi_skew               0.076050
tavg_skew               0.076124
o3_skew                 0.076042
aqhi_max                0.072524
prcp_max                0.072524
wspd_max                0.072524
ndvi_max                0.072524
tavg_max                0.072524
o3_max                  0.072524
region                  0.000000
dtype: float64

***
### FEATURE ENGINEERING

### From mouvements to sensors data

- Idea is to get the location of the hive at time X between June 1^st^ and August 31 
- Once we have that info, we simply do statistics on the sequence 

In [ ]:
if RUN_PREPROCESS:
    mov = mov[mov['hive_identity_id'].isin(list(data.hive_identity_id.unique()))].reset_index(drop=True)

In [23]:
if RUN_PREPROCESS:
    # WARNING THIS TAKES A WHILE (15-30 minutes...or so)
    mov = (
        mov.assign(
            time=[pd.date_range(start, end) for start, end in zip(mov['from_when'], mov['to_when'])]
        )
        .explode('time', ignore_index=True)
    )

In [ ]:
if RUN_PREPROCESS:
    mov['time'] = pd.to_datetime(mov['time']).dt.date
    mov = mov.loc[pd.to_datetime(mov['time']).dt.month.isin(FEATURES_MONTH)].reset_index(drop=True)

In [24]:
if RUN_PREPROCESS:
    merged = pd.merge(mov, aqi, on=["yard_id", "time"], how="left")
    merged = pd.merge(merged, weather, on=["yard_id", "time"], how="left")
    merged = pd.merge(merged, ee, on=["yard_id", "time"], how="left")
    merged.sort_values("time", ascending=True, inplace=True)
    # Keep value constant if no change 
    merged.fillna(method="bfill", inplace=True)
    
    merged.head()

In [25]:
if RUN_PREPROCESS:
    merged.isnull().mean()

TSun and Wdir high null, just ignore them 

In [26]:
if RUN_PREPROCESS:
    merged.drop(['tsun', 'wdir'], axis=1, inplace=True)

### Compute stats for each hid

In [27]:
if RUN_PREPROCESS:
    merged['season'] = pd.to_datetime(merged['time']).dt.year

In [28]:
if RUN_PREPROCESS:
    # This takes 10 min or so locally
    # merged['month'] = pd.to_datetime(merged['time']).dt.month
    
    averages = (
        merged
        .drop(['from_when', 'to_when', 'time', 'submitted_at'], axis=1, errors="ignore")
        .groupby(["hive_identity_id", "season"])
        .mean()
        .reset_index()
    )
    
    skewed = (
        merged
        .drop(['from_when', 'to_when', 'time', 'submitted_at'], axis=1, errors="ignore")
        .groupby(["hive_identity_id", "season"])
        .agg(lambda x: skew(x))
        .reset_index()
    )
    
    maxed = (
        merged
        .drop(['from_when', 'to_when', 'time', 'submitted_at'], axis=1, errors="ignore")
        .groupby(["hive_identity_id", "season"])
        .max()
        .reset_index()
    )
    
    mode = (
        merged
        .drop(['from_when', 'to_when', 'time', 'submitted_at'], axis=1, errors="ignore")
        .groupby(["hive_identity_id", "season"])
        .mode()
        .reset_index()
    )

In [ ]:
if RUN_PREPROCESS:
    averages.rename(
        columns={
            "aqhi": "aqhi_average",
            "prcp": "prcp_average",
            "wspd": "wspd_average",
            "ndvi": "ndvi_average",
            "tavg": "tavg_average",
            "o3": "o3_average",
            "hive_identity_id": "hid"
        },
        inplace=True
    )
    
    skewed.rename(
        columns={
            "aqhi": "aqhi_skew",
            "prcp": "prcp_skew",
            "wspd": "wspd_skew",
            "ndvi": "ndvi_skew",
            "tavg": "tavg_skew",
            "o3": "o3_skew",
            "hive_identity_id": "hid"
        },
        inplace=True
    )
    
    maxed.rename(
        columns={
            "aqhi": "aqhi_max",
            "prcp": "prcp_max",
            "wspd": "wspd_max",
            "ndvi": "ndvi_max",
            "tavg": "tavg_max",
            "o3": "o3_max",
            "hive_identity_id": "hid"
        },
        inplace=True
    )

    # Uncomment if needed
    # mode.rename(
    #     columns={
    #         "aqhi": "aqhi_max",
    #         "prcp": "prcp_max",
    #         "wspd": "wspd_max",
    #         "ndvi": "ndvi_max",
    #         "tavg": "tavg_max",
    #         "o3": "o3_max",
    #         "hive_identity_id": "hid"
    #     },
    #     inplace=True
    # )

In [ ]:
if RUN_PREPROCESS:
    data.rename(columns={"hive_identity_id":"hid"}, inplace=True)

### Bring remote sensors with season state (merge)

In [ ]:
if RUN_PREPROCESS:
    print(len(data))
    
    data = pd.merge(
        data,
        averages[['season', 'hid', 'aqhi_average', 'prcp_average', 'wspd_average',
                  'ndvi_average', 'tavg_average', 'o3_average']],
        on=["season", "hid"],
        how="left"
    )
    print(len(data))
    
    data = pd.merge(
        data,
        skewed[['season', 'hid', 'aqhi_skew', 'prcp_skew', 'wspd_skew',
                'ndvi_skew', 'tavg_skew', 'o3_skew']],
        on=["season", "hid"],
        how="left"
    )
    print(len(data))
    
    data = pd.merge(
        data,
        maxed[['season', 'hid', 'aqhi_max', 'prcp_max', 'wspd_max',
               'ndvi_max', 'tavg_max', 'o3_max']],
        on=["season", "hid"],
        how="left"
    )
    print(len(data))
    
    # Uncomment if needed
    # data = pd.merge(
    #     data,
    #     mode[['season', 'hid', 'aqhi_max', 'prcp_max', 'wspd_max',
    #           'ndvi_max', 'tavg_max', 'o3_max']],
    #     on=["season", "hid"],
    #     how="left"
    # )

In [29]:
data.head()

,hid,creation_date,operation_id,death_date,season,season_start_month,season_start_day,death_next_season,hive_age_next_season,aqhi_average,...,ndvi_skew,tavg_skew,o3_skew,aqhi_max,prcp_max,wspd_max,ndvi_max,tavg_max,o3_max,region
0,36865,2021-09-20,55,2022-05-14,2021,5,14,False,236,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QC
1,16649,2021-05-19,55,2022-05-14,2021,5,14,False,360,2.533632,...,0.297063,-0.510858,0.687366,6.0,34.6,25.0,0.460514,26.7,104.845833,QC
2,14267,2021-05-08,55,2022-05-14,2021,5,14,False,371,2.602740,...,0.216794,-0.395353,0.657954,6.0,34.6,25.0,0.449957,27.2,104.845833,QC
3,16020,2021-05-17,55,2022-05-14,2021,5,14,False,362,2.580488,...,0.301466,-0.454580,0.719820,6.0,34.6,25.0,0.460514,26.7,104.845833,QC
4,16923,2021-05-25,55,2022-05-14,2021,5,14,False,354,1.946341,...,0.233874,-0.134386,0.669923,5.0,34.6,25.0,0.477245,26.7,88.303750,QC


In [30]:
len(data)

133584

### Add region

In [32]:
# if RUN_PREPROCESS:
operation_meta = pd.read_csv(os.path.join(ROOT_PATH,"ops_meta_loc.csv"))
operation_meta.index = operation_meta['Operation ID']
regions = operation_meta.to_dict(orient="dict")['Location']

In [33]:
regions

{193: 'Woodland, CA',
 192: 'Jennings, LA',
 69: 'Hay Lakes, AB',
 84: 'Turtle Lake, ND',
 167: 'Billerica, MA',
 55: 'Mirabel, QC',
 153: 'Coaldale, AB',
 51: 'Naches, WA',
 207: 'Myakka, FL',
 161: 'Saint-Sylvestre, QC',
 194: 'Worland, WY',
 87: 'Peace River, AB',
 208: 'Arbuckle, CA',
 210: 'Saint-Stanislas-de-Kostka, QC',
 160: 'Stewiacke, NS',
 195: 'Palo Cedro, CA',
 199: 'Hillsboro, KS',
 205: 'Fort Vermilion, AB',
 212: 'Chateaugay, NY',
 36: 'Spokane, WA',
 218: 'Youngsville, LA',
 45: 'Newcastle, CA',
 177: 'Jamestown, ND',
 159: 'Sherbrooke, QC',
 83: 'Montreal, QC'}

In [34]:
def try_region(x):
    try:
        return regions[int(x)].split(",")[1]
    except:
        return 'Unknown'

In [35]:
if RUN_PREPROCESS:
    data['region'] = data['operation_id'].apply(lambda x: try_region(x))
    data.groupby(['region', 'season']).count()

### Saving preprocess data

In [ ]:
if RUN_PREPROCESS:
    data.to_csv(os.path.join(ROOT_PATH, "preprocess.csv"), index=False)
else:
    data = pd.read_csv(os.path.join(ROOT_PATH, "preprocess.csv"))